In [ ]:
import tensorflow
from tensorflow import keras
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

data1=pd.read_csv('IPL Ball-by-Ball 2008-2020.csv')
data2=pd.read_csv('IPL Matches 2008-2020.csv')

#less than 3 percent matches contained D/L and we can remove those mathes from our dataset as it may result in 
id_to_drop=list(data2['id'][data2['method']=='D/L'])

matches=data2[~data2['id'].isin(id_to_drop)]

deliveries=data1[~data1['id'].isin(id_to_drop)]

#Now we have removed the D/L matches from our dataset

unwanted_features_matches=['player_of_match','result','result_margin','method','umpire1','umpire2']

matches_filtered=matches.drop(columns=unwanted_features_matches,axis=1)

train_df = deliveries.groupby(['id', 'inning', 'over', 'ball','batting_team','bowling_team' ])[['total_runs', 'is_wicket']].agg(['sum']).reset_index()
train_df.columns = train_df.columns.get_level_values(0)

train_df['innings_wickets'] = train_df.groupby(['id', 'inning'])['is_wicket'].cumsum()
train_df['innings_score'] = train_df.groupby(['id', 'inning'])['total_runs'].cumsum()
train_df=train_df.drop(['is_wicket'],axis=1)

train_df=train_df.drop(['total_runs'],axis=1)

deliveries_filtered=deliveries.drop(['batsman_runs','extra_runs','total_runs','non_boundary','is_wicket','dismissal_kind','fielder','extras_type','batting_team','bowling_team'],axis=1)

deliveries_final=pd.merge(deliveries_filtered,train_df,on=['id','inning','over','ball'])

data=pd.merge(deliveries_final,matches_filtered,on=['id'])
data=data.drop(['player_dismissed','city'],axis=1)

data=data.drop(['team1','team2'],axis=1)

# data.to_csv("final_ipl_dataset",index=False)

input=pd.read_csv('final_ipl_dataset')
l=list(range(335982,336041))
l

data['date'] = pd.to_datetime(data['date'])

data = data[data['date'].dt.year >= 2014]

drops=list(data[data['id']==829813])

data=data[data['id']!=829813]

data['eliminator'].replace({'N' : 0,'Y' : 1},inplace=True)

data[data['winner'].isnull()]

data=data[data['id']!=1178424]

data['eliminator']=data['eliminator'].astype('int32')

sorted_data=data.sort_values(by=['id','inning','over','ball'],ascending=True)


x = data1.groupby(['batsman', 'inning', 'bowling_team'])['batsman_runs'].sum().to_frame()

# Resetting the index to include 'batsman', 'inning', and 'bowling_team' as columns
x.reset_index(inplace=True)


pd.merge(x,sorted_data,on=['batsman','inning','bowling_team'])

target=data1.groupby(['id','inning'])['total_runs'].sum().to_frame()
target.reset_index(inplace=True)

target=target[target['inning']==1]

target['inning']=target['inning'].replace(1,2)

target.rename(columns={'total_runs':'target_score'},inplace=True)


wickets = data1.groupby(['bowler', 'inning', 'batting_team'])['is_wicket'].sum().to_frame()

# Resetting the index to include 'batsman', 'inning', and 'bowling_team' as columns
wickets.reset_index(inplace=True)
#main thing starting


y=pd.merge(sorted_data,x,on=['batsman','inning','bowling_team'])

final=pd.merge(y,wickets,on=['bowler', 'inning', 'batting_team'])


final=final.replace({'bat':1,'field':0})
final=final.sort_values(by=['id','inning','over','ball'],ascending=True)

#including runs remaining,current run rate,required run rate

final=pd.merge(final,target,on=['id', 'inning'],how='left')

final['target_score']=np.where(final['inning']==1,-1,final['target_score'])


final['remaining_score']=final['target_score']-final['innings_score']

final['remaining_score']=np.where(final['inning']==1,-1,final['remaining_score'])

final['remaining_balls']=120-((final['over']*6)+final['ball'])

# final['current_runrate']=final['innings_score']/((final['over']*6)+final['ball'])
# final


# final['required_runrate']=final['target_score']/120
# final

# final['required_runrate']=np.where(final['required_runrate']<1,-1,final['required_runrate'])
# final

# final['current_runrate']=np.where(np.isnan(final['current_runrate']),0,final['current_runrate'])
# final

final['new'] = np.where(final['winner'] == final['batting_team'], 1, 0)
# final['new'] = np.where(final['winner'] == final['toss_winner'], 1, 0)


# final['neutral_venue']=final['neutral_venue'].replace(1,0)
# final['neutral_venue'].value_counts()

# #Converting value of every column with respect to battin team column
# # final['neutral_venue'] = np.where(final['batting_team'] == final['winner'], 1, 0)
# final['neutral_venue'] = np.where(final['toss_winner'] == final['winner'], 1, 0)
# final

# final.rename(columns={'neutral_venue': 'venue_advantage'}, inplace=True)

unwanted=['id','batsman','non_striker','bowler','date','winner','venue','neutral_venue','toss_decision','toss_winner']
# final.drop(unwanted,axis=1,inplace=True)

# test_match = pd.get_dummies(final, columns = ['batting_team','bowling_team','toss_winner'])

test_match = pd.get_dummies(final, columns = ['batting_team','bowling_team'])

test_match=test_match[test_match['id']==1082599]
test_match.drop(unwanted,axis=1,inplace=True)

final=final[final['id']!=1082599]

# # test_match = pd.get_dummies(final, columns = ['batting_team','bowling_team','venue','toss_winner'])
# # print(test_match)
# test_match = pd.get_dummies(final, columns = ['batting_team','bowling_team','venue','toss_winner'])
# print(test_match)
final.drop(unwanted,axis=1,inplace=True)


# one_hot_encoded_data = pd.get_dummies(final, columns = ['batting_team','bowling_team','venue','toss_winner'])
one_hot_encoded_data = pd.get_dummies(final, columns = ['batting_team','bowling_team'])

# X=one_hot_encoded_data.drop(['new','toss_decision'],axis=1)
#see if removing neutral venue has some effect
X=one_hot_encoded_data.drop(['new'],axis=1)
Y=one_hot_encoded_data['new']

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=42)

import tensorflow as tf
from tensorflow import keras
#This is the original model
# model=keras.Sequential([
#     keras.layers.Dense(35,input_shape=(35,),activation='relu'),
#     keras.layers.Dense(30,activation='relu'),
#     keras.layers.Dense(18,activation='relu'),
#     keras.layers.Dense(1,activation='sigmoid')
# ])

model=keras.Sequential([
    keras.layers.Dense(35,input_shape=(35,),activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(30,activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(18,activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.fit(x_train,y_train,epochs=10)

model.evaluate(x_test,y_test)

# y_pred=model.predict(x_test)

# # test_match.drop(['toss_decision','new'],axis=1,inplace=True)

# test_match.drop(['new'],axis=1,inplace=True)

# prediction=model.predict(test_match)
# prediction

# i=0
# while(True):
#     if(prediction[i]>0.5):
#         break
#     i=i+1
# i

# prediction=prediction*100

# X_values=range(0,len(prediction))

# prediction[60:100]

# plt.plot(X_values,prediction)

# test_match.iloc[60:100,0:15]

# test_match.columns

y_pred=model.predict(x_test)

# test_match.drop(['toss_decision','new'],axis=1,inplace=True)

test_match.drop(['new'],axis=1,inplace=True)

prediction=model.predict(test_match)
prediction

# i=0
# while(True):
#     if(prediction[i]>0.5):
#         break
#     i=i+1
# i

prediction=prediction*100

X_values=range(0,len(prediction))

plt.plot(X_values,prediction)

# test_match.iloc[60:100,0:15]

# test_match.columns
